<a href="https://colab.research.google.com/github/Danil-Kazakov/Data-HW9/blob/main/Data-HW9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from kerastuner.tuners import RandomSearch


<ipython-input-5-62b396b6b306>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


Підготовка даних та моделi

In [6]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))


4422102/4422102 [==============================] - 0s 0us/step


Навчання та оцінка моделі

In [7]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=128, verbose=2)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Точність на тестових даних: {test_acc * 100:.2f}%')


Epoch 1/20
469/469 - 54s - loss: 0.9058 - accuracy: 0.6636 - val_loss: 0.5636 - val_accuracy: 0.7918 - 54s/epoch - 115ms/step
Epoch 2/20
469/469 - 52s - loss: 0.5923 - accuracy: 0.7840 - val_loss: 0.4706 - val_accuracy: 0.8269 - 52s/epoch - 112ms/step
Epoch 3/20
469/469 - 51s - loss: 0.5137 - accuracy: 0.8170 - val_loss: 0.4317 - val_accuracy: 0.8385 - 51s/epoch - 108ms/step
Epoch 4/20
469/469 - 53s - loss: 0.4676 - accuracy: 0.8323 - val_loss: 0.4215 - val_accuracy: 0.8450 - 53s/epoch - 113ms/step
Epoch 5/20
469/469 - 51s - loss: 0.4387 - accuracy: 0.8419 - val_loss: 0.3704 - val_accuracy: 0.8651 - 51s/epoch - 109ms/step
Epoch 6/20
469/469 - 51s - loss: 0.4182 - accuracy: 0.8516 - val_loss: 0.3596 - val_accuracy: 0.8671 - 51s/epoch - 108ms/step
Epoch 7/20
469/469 - 51s - loss: 0.4001 - accuracy: 0.8551 - val_loss: 0.3464 - val_accuracy: 0.8687 - 51s/epoch - 108ms/step
Epoch 8/20
469/469 - 50s - loss: 0.3810 - accuracy: 0.8619 - val_loss: 0.3360 - val_accuracy: 0.8770 - 50s/epoch - 107

In [11]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units_1', min_value=128, max_value=512, step=128), activation='relu', input_shape=(28 * 28,)))
    model.add(Dropout(hp.Float('dropout_1', 0.2, 0.5, step=0.1)))
    model.add(Dense(hp.Int('units_2', min_value=128, max_value=512, step=128), activation='relu'))
    model.add(Dropout(hp.Float('dropout_2', 0.2, 0.5, step=0.1)))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='my_dir', project_name='fashion_mnist_dense')

tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))


Trial 5 Complete [00h 02m 25s]
val_accuracy: 0.8669000069300333

Best val_accuracy So Far: 0.8740333318710327
Total elapsed time: 00h 16m 34s
